In [ ]:
import pandas as pd

import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# Load datasets
train_df = pd.read_csv("/kaggle/input/cs-985-6-spotify-classification-problem-2025/CS98XClassificationTrain.csv")
test_df = pd.read_csv("/kaggle/input/cs-985-6-spotify-classification-problem-2025/CS98XClassificationTest.csv")

# Drop missing values in target column
train_df.dropna(subset=["top genre"], inplace=True)

# Save test IDs
test_ids = test_df["Id"]

# Drop unnecessary columns
drop_cols = ["Id", "title", "artist"]
train_df.drop(columns=drop_cols, inplace=True)
test_df.drop(columns=drop_cols, inplace=True)

# Encode target variable
label_encoder = LabelEncoder()
train_df["top genre"] = label_encoder.fit_transform(train_df["top genre"])

# Remove rare genres (less than 3 occurrences)
genre_counts = train_df["top genre"].value_counts()
train_df = train_df[train_df["top genre"].isin(genre_counts[genre_counts >= 3].index)]

# Separate features and target variable
X, y = train_df.drop(columns=["top genre"]), train_df["top genre"]
X = X.select_dtypes(include=[np.number]).copy()
test_df = test_df.select_dtypes(include=[np.number]).copy()


# Handle missing values
imputer = SimpleImputer(strategy="median")
X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
test_df = pd.DataFrame(imputer.transform(test_df), columns=test_df.columns)

# Normalize features
#scaler = StandardScaler()
#X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
#test_df = pd.DataFrame(scaler.transform(test_df), columns=test_df.columns)

# Feature selection
X_selected = X.values
test_df_selected = test_df.values


# Dynamic Stratified K-Fold to prevent errors
n_splits = min(3, y.value_counts().min())
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Optuna Hyperparameter Optimization
def objective(trial):
    param = {
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
        "objective": "multiclass",
        "num_class": len(np.unique(y)),
        "metric": "multi_logloss",
        "boosting_type": "gbdt",
        "max_depth": trial.suggest_int("max_depth", 10, 50),
        "num_leaves": trial.suggest_int("num_leaves", 20, 200),
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.2),
        "n_estimators": trial.suggest_int("n_estimators", 500, 2000),
        "subsample": trial.suggest_float("subsample", 0.7, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.7, 1.0),
        "class_weight": "balanced"
    }
    
    accuracies = []
    
    for train_idx, val_idx in skf.split(X_selected, y):
        X_train, X_valid = X_selected[train_idx], X_selected[val_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[val_idx]

        model = lgb.LGBMClassifier(**param, verbose=-1)
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric="multi_logloss",
            callbacks=[lgb.early_stopping(100)]
        )
        
        y_pred_valid = model.predict(X_valid)
        accuracies.append(accuracy_score(y_valid, y_pred_valid))
    
    #  Print validation accuracy for each trial
    print(f"Trial {trial.number} Accuracy: {np.mean(accuracies):.4f}")
    
    return np.mean(accuracies)

# Run Optuna for better hyperparameters
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)  # Increased trials for better tuning

#  Print Best Validation Accuracy
print(f"\n Best Validation Accuracy After Optuna: {study.best_value:.4f}")

# Train final LightGBM model with best hyperparameters
best_params = study.best_params
final_model = lgb.LGBMClassifier(**best_params, verbose=-1)
final_model.fit(X_selected, y)

# Predict genres for the test dataset
y_pred_test = final_model.predict(test_df_selected)

# Convert predictions back to original genre labels
y_pred_test_labels = label_encoder.inverse_transform(y_pred_test)

# Prepare & save submission (Kaggle download path)
submission = pd.DataFrame({"Id": test_ids, "top genre": y_pred_test_labels})
out_path = "/kaggle/working/spotify_genre_predictions.csv"
submission.to_csv("/kaggle/working/spotify_genre_predictions.csv", index=False)

print("Saved:", out_path)



[I 2025-09-25 03:02:04,913] A new study created in memory with name: no-name-6195ccf8-b09c-4ad9-b1ca-09fb155b27d5


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[52]	valid_0's multi_logloss: 2.65382
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[68]	valid_0's multi_logloss: 2.58462
Training until validation scores don't improve for 100 rounds


[I 2025-09-25 03:02:05,994] Trial 0 finished with value: 0.1899626517273576 and parameters: {'min_child_samples': 81, 'max_depth': 17, 'num_leaves': 105, 'learning_rate': 0.08220816316628564, 'n_estimators': 1584, 'subsample': 0.8672877451542771, 'colsample_bytree': 0.9033358554132603}. Best is trial 0 with value: 0.1899626517273576.


Early stopping, best iteration is:
[84]	valid_0's multi_logloss: 2.59394
Trial 0 Accuracy: 0.1900
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[54]	valid_0's multi_logloss: 2.62009
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[57]	valid_0's multi_logloss: 2.51509
Training until validation scores don't improve for 100 rounds


[I 2025-09-25 03:02:07,256] Trial 1 finished with value: 0.2234593837535014 and parameters: {'min_child_samples': 60, 'max_depth': 10, 'num_leaves': 126, 'learning_rate': 0.08435547279759671, 'n_estimators': 1410, 'subsample': 0.9542868090355671, 'colsample_bytree': 0.7302616058531445}. Best is trial 1 with value: 0.2234593837535014.


...

Early stopping, best iteration is:
[79]	valid_0's multi_logloss: 2.50167
Trial 94 Accuracy: 0.3350
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[50]	valid_0's multi_logloss: 2.46765
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[50]	valid_0's multi_logloss: 2.42089
Training until validation scores don't improve for 100 rounds


[I 2025-09-25 03:09:14,185] Trial 95 finished with value: 0.3100373482726424 and parameters: {'min_child_samples': 22, 'max_depth': 46, 'num_leaves': 38, 'learning_rate': 0.05661890959544566, 'n_estimators': 774, 'subsample': 0.8746309099108877, 'colsample_bytree': 0.7377020366640367}. Best is trial 23 with value: 0.34073295985060686.


Early stopping, best iteration is:
[48]	valid_0's multi_logloss: 2.49737
Trial 95 Accuracy: 0.3100
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[105]	valid_0's multi_logloss: 2.42599
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[111]	valid_0's multi_logloss: 2.41755
Training until validation scores don't improve for 100 rounds


[I 2025-09-25 03:09:17,607] Trial 96 finished with value: 0.29332399626517275 and parameters: {'min_child_samples': 27, 'max_depth': 48, 'num_leaves': 74, 'learning_rate': 0.030215944371758553, 'n_estimators': 1120, 'subsample': 0.9250749630026301, 'colsample_bytree': 0.7710518512181029}. Best is trial 23 with value: 0.34073295985060686.


Early stopping, best iteration is:
[109]	valid_0's multi_logloss: 2.4887
Trial 96 Accuracy: 0.2933
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[127]	valid_0's multi_logloss: 2.34674
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[118]	valid_0's multi_logloss: 2.37995
Training until validation scores don't improve for 100 rounds
